# Generating microstructures using Dream3D
First import gen_microstructures function from src package

In [37]:
import os
from pathlib import Path
import numpy as np
from src.generate_microstructures import gen_microstructures, print_params

In [38]:
# Get name of directory that contains the PRISMS-Fatigue scripts
DIR_LOC = os.path.join(os.path.dirname(os.getcwd()), "src") 
DIR_LOC

'E:\\OneDrive\\UWM\\William_Musinski__Surana\\Research_Code\\Research\\src'

Next set up parameters for microstructure generating:

In [40]:
# Number of microstructure instantiations to generate using DREAM.3D
num_instantiations = 5

In [42]:
''' Specify directories and paths '''
# Directory where microstructure data should be generated and pre-processed
# This command creates a directory in the same location as the "PRISMS-Fatigue" directory with python scripts and DREAM.3D files
directory = os.path.dirname(DIR_LOC) + '\\generated_microstructures\\'

# Alternatively, the directory can be expressed as an absolute path as:
# directory = r'C:\Users\stopk\Documents\GitHub\PRISMS-Fatigue\tutorial\test_run_1'

# Location of DREAM.3D input file; should consist of only the "StatsGenerator" and "Write DREAM.3D Data File"
# "StatsGenerator" inputs include grain size distribution, crystallographic texture, grain morphology, etc.
# Six ".dream3d" files are included in PRISMS-Fatigue
d3d_input_file = os.path.abspath(DIR_LOC) + '\\Al7075_random_texture_equiaxed_grains.dream3d'

# Once again, this may be specified using an absolute path as:
# d3d_input_file = r'C:\Users\stopk\Documents\GitHub\PRISMS-Fatigue\Al7075_cubic_texture_equiaxed_grains.dream3d'

# Average grain size as determined in the "StatsGenerator" filter of the .dream3d file above
# Used for automated band and sub-band sizing as shown below but which can be overwritten by the user
# Therefore, this does NOT change the grain size generated and only affects the way in which microstructures are banded and sub-banded!
avg_grain_size = 0.014 # millimeters

# Location of DREAM.3D .json pipeline
# This can be modified by the user to include additional outputs
d3d_pipeline_path = os.path.abspath(DIR_LOC) + '\\Dream3D_microstructure_pipeline.json'

# Once again, this may be specified using an absolute path as:
# d3d_pipeline_path = r'C:\Users\stopk\Documents\GitHub\PRISMS-Fatigue\Dream3D_microstructure_pipeline.json'

# Location of DREAM.3D 'PipelineRunner.exe' file; this should be in the DREAM.3D program folder
d3d_executable_path = '"' + os.path.abspath(Path(DIR_LOC).parents[0]) + '\\Dream3D-build\\DREAM3D-6.5.171-Win64\\PipelineRunner.exe"'


''' Specify desired microstructure size and shape '''
# Size of microstructure instantiations in millimeters, in the X, Y, and Z directions, respectively.
size  = np.asarray([.0725,.0725,.0725])

# Shape of microstructure instantiations (number of voxels/elements), in the X, Y, and Z directions, respectively.
# IMPORTANT: at this point, only CUBIC voxel functionality supported even with a non-cubic microstructure
# I.e., size = [.05, .1, .025] and shape = [50, 100, 25] is acceptable
shape = np.asarray([29,29,29])

''' Specify details of banding and sub-banding process '''
# Specify the number of elements in each sub-band for volume averaging
# Please see the references below for more information
# WARNING!: If this is too low for very refined grains, this module will take a long time to run!
# This is because it attempts to determine all UNIQUE combinations of some number of neighboring elements

# NOTE: Aim for ~8-10% of the average grain volume as the num_vox, as specified below
# This is especially important when comparing microstructures with different grain sizes!
num_vox_percentage = 0.10

# This line calculates num_vox to be "num_vox_percentage" percent of the predicted average number of elements per grain
num_vox = np.around(np.prod(shape) / (np.prod(size) / ( (1.0/6.0) * np.pi * avg_grain_size ** 3  ) ) * num_vox_percentage).astype(int)

# Comment out the above line and uncomment the line below to manually set the number of elements per sub-band
# num_vox = 8

# Specify the thickness of bands in terms of number of elements
# Ideally, this should result in approximately 6 bands for coarser grains (~100 elements per grain)
# NOTE: This should also be ~one or a few micrometers in thickness based on experimental observations of slip bands
# See reference below, Castelluccio and McDowell

# This line calculates the band thickness IN MULTIPLES OF element thickness
band_thickness = np.around( avg_grain_size / ( 6 * size[0]/shape[0]) )
if band_thickness < 1:
    # The microstructure is relatively coarse so need to override band thickness to 1 element in width
    print('Setting band thickness to 1 element in width')
    band_thickness = 1.0

# Comment out the above line and uncomment the line below to manually set the band element thickness
# Set as float for subsequent calculations
# band_thickness = 3.0

# Number of crystallographic slip planes for FIP Volume averaging
# There are four slip planes in the Al 7075-T6 material system (see references below)
num_planes = 4

# Specify whether bands in grains should be further assigned to unique sub-bands
# WARNING!: this is slow for a) microstructures with many grains, and 2) very fine grains, i.e., thousands of elements per grain
# NOTE: Microstructure(s) can be created with this intially set to False, and the "gen_microstructures" can be executed with the variable "generate_new_microstructure_files" set to False, so that the SAME microstructure(s) can undergo the sub-banding process at a later time.
create_sub_bands = True

# Specify whether the centroids of grains split by the microstructure boundary should be manipulated to reflect periodicity
# Setting this to False will substantially increase the speed at which microstructures are pre-processed, but will reduce the accuracy of computed and volume-averaged fatigue indicator parameters downstream!
# Users can set this to False but rerun this script with "generate_new_microstructure_files" set to False and "compute_kosher_grain_centroids" set to True to compute the realistic grain centroids at a later time
compute_kosher_grain_centroids = True

''' Specify boundary conditions '''
# Boundary conditions, either "periodic" or "free surface", for X,Y,Z directions
# Three possible combinations: 1) all 'periodic', 2) all 'free', or 3) two 'periodic' + one 'free'
face_bc = ['periodic', 'periodic', 'periodic']

# Specify whether DREAM.3D was previously executed on these files
# If set to False, the script will NOT generate new DREAM.3D microstructure(s) and instead process the existing microstructures by reading the .csv and GrainID_#.txt files
# Reasons to set this to False:
#     1) Process the same set of microstructures with a different number of elements per sub-band, and store these in a separate folder. In this case, copy over the .csv and GrainID_#.txt files to a new folder and run this script.
#     2) Generate a set of ['periodic', 'periodic', 'periodic'] microstructures, and then reprocesses them with one set of faces set to non-periodic, i.e., ['periodic', 'free', 'periodic'], to study bulk vs. surface fatigue effects.
# NOTE: If the variable is set to False, all that is needed in the folder is the DREAM.3D exported .csv and "grainID.txt" files for each microstructure
generate_new_microstructure_files = True

# Print the parameters of this microstructure set to a text file
print_params(directory, size, shape, face_bc, num_vox, band_thickness, num_planes, num_instantiations, d3d_input_file)
directory, size, shape, face_bc, num_vox, band_thickness, num_planes, num_instantiations, d3d_input_file

('E:\\OneDrive\\UWM\\William_Musinski__Surana\\Research_Code\\Research\\generated_microstructures\\',
 array([0.0725, 0.0725, 0.0725]),
 array([29, 29, 29]),
 ['periodic', 'periodic', 'periodic'],
 9,
 1.0,
 4,
 5,
 'E:\\OneDrive\\UWM\\William_Musinski__Surana\\Research_Code\\Research\\src\\Al7075_random_texture_equiaxed_grains.dream3d')

In [43]:
# Call to the main function
gen_microstructures(directory, size, shape, face_bc, num_vox, band_thickness, num_planes, create_sub_bands, num_instantiations, generate_new_microstructure_files, compute_kosher_grain_centroids, d3d_input_file, d3d_pipeline_path, d3d_executable_path)

Current output microstructure file: E:/OneDrive/UWM/William_Musinski__Surana/Research_Code/Research/generated_microstructures//Output_FakeMatl_0.vtk
Current output microstructure file: E:/OneDrive/UWM/William_Musinski__Surana/Research_Code/Research/generated_microstructures//Output_FakeMatl_1.vtk
Current output microstructure file: E:/OneDrive/UWM/William_Musinski__Surana/Research_Code/Research/generated_microstructures//Output_FakeMatl_2.vtk
Current output microstructure file: E:/OneDrive/UWM/William_Musinski__Surana/Research_Code/Research/generated_microstructures//Output_FakeMatl_3.vtk
Current output microstructure file: E:/OneDrive/UWM/William_Musinski__Surana/Research_Code/Research/generated_microstructures//Output_FakeMatl_4.vtk
Band width of 0.0025
Calculate realistic grain centroids

Finished 0.0% grains
Finished 37.9% grains
Finished 75.8% grains
Time to determine kosher element centroids: 0.72 seconds 
Amount of bands in total is 5263
Time to band microstructure: 0.19 seconds